In [1]:
%matplotlib inline
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
import numpy as np 
import pandas as pd 
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from ast import literal_eval
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer

import warnings; warnings.simplefilter('ignore')

def weighted_rating(x, m, C):
    v = x['vote_count']
    R = x['vote_average']
    return (v/(v+m) * R) + (m/(m+v) * C)

def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

def filter_keywords(x, s):
    words = []
    for i in x:
        if i in s:
            words.append(i)
    return words

cols = ['id', 'movieId', 'title', 'genres', 'description', 'keywords', \
        'cast', 'director', 'spoken_languages', 'production_companies',\
        'production_countries', 'popularity', 'year', 'vote_average',\
         'vote_count', 'wr']

def read_dataset(metadata_path, links_small_path, credits_path,keywords_path):
    meta = pd.read_csv(metadata_path)
    links_small = pd.read_csv(links_small_path)
    credits = pd.read_csv(credits_path)
    keywords = pd.read_csv(keywords_path)
    return meta, links_small, credits, keywords

def cal_weighted_rating(meta, percentile=0.95):
    vote_counts = meta[meta['vote_count'].notnull()]['vote_count'].astype('int')
    vote_averages = meta[meta['vote_average'].notnull()]['vote_average'].astype('int')
    C = vote_averages.mean()
    m = vote_counts.quantile(percentile)

    meta = meta[(meta['vote_count'].notnull()) \
                     & (meta['vote_average'].notnull())]
    meta['vote_count'] = meta['vote_count'].astype('int')
    meta['vote_average'] = meta['vote_average'].astype('int')
    meta['wr'] = meta.apply(lambda x: weighted_rating(x, m, C), axis=1)

    return meta
    
def movie_feature(metadata_path, links_small_path, credits_path,keywords_path, \
                  percentile=0.95, more_weight_on = None, \
                    stemmer = SnowballStemmer('english'), cols=cols):
    # read dataset
    meta, links_small, credits, keywords = read_dataset(metadata_path, \
        links_small_path, credits_path,keywords_path)

    # change type + drop
    links_small = links_small[links_small['tmdbId'].notnull()]
    links_small['tmdbId'] = links_small['tmdbId'].astype('int')

    meta = meta.drop([19730, 29503, 35587])
    meta['popularity'] = meta[meta['popularity'].notnull()]['popularity'].astype('float')
    meta['id'] = meta['id'].astype('int')
    meta['year'] = pd.to_datetime(meta['release_date'], errors='coerce').apply(\
        lambda x: str(x).split('-')[0] if x != np.nan else np.nan)
    meta = meta[meta.production_companies.notnull()]
    keywords['id'] = keywords['id'].astype('int')
    credits['id'] = credits['id'].astype('int')

    # calcualte weighted rating for movies
    meta = cal_weighted_rating(meta)

    # merge meta + link small => create a smaller dataset for recommend
    smd = meta[meta['id'].isin(links_small['tmdbId'])]

    # create description feature
    smd['tagline'] = smd['tagline'].fillna('')
    smd['description'] = smd['overview'] + smd['tagline']
    smd['description'] = smd['description'].fillna('')
    
    # merge credit + keywords + links_small
    smd = smd.merge(credits, on='id')
    smd = smd.merge(keywords, on='id')
    smd = smd.merge(links_small, left_on='id', right_on='tmdbId')

    # feature engineering
    literal_features = ['cast', 'spoken_languages', 'genres', 'keywords',\
                        'production_companies', 'production_countries']
    for fearture in literal_features:
        smd[fearture] = smd[fearture].apply(literal_eval)
        smd[fearture] = smd[fearture].apply(lambda x: [i['name'] \
                                            for i in x] if isinstance(x, list) else [])
    smd['crew'] = smd['crew'].apply(literal_eval)
    smd['director'] = smd['crew'].apply(get_director)

    # top 3 actors
    smd['cast'] = smd['cast'].apply(lambda x: x[:3] if len(x) >=3 else x)
   
    # Strip Spaces and Convert to Lowercase 
    smd['cast'] = smd['cast'].apply(lambda x: [str.lower(\
        i.replace(" ", "")) for i in x])
    smd['director'] = smd['director'].astype('str').apply(\
        lambda x: str.lower(x.replace(" ", "")))
    if more_weight_on:
        smd[more_weight_on] = smd[more_weight_on].apply(lambda x: [x,x,x])
    
    # choose keywords appear more than once + stemming
    s = smd.apply(lambda x: pd.Series(x['keywords']),axis=1).stack().reset_index(level=1, drop=True)
    s = s.value_counts()
    s = s[s > 1]
    smd['keywords'] = smd['keywords'].apply(lambda x: filter_keywords(x, s))
    smd['keywords'] = smd['keywords'].apply(lambda x: [stemmer.stem(i) for i in x])
    smd['keywords'] = smd['keywords'].apply(lambda x: \
                                            [str.lower(i.replace(" ", "")) for i in x])

    return smd[cols]
    


In [32]:
credits_ = r'/kaggle/input/the-movies-dataset/credits.csv'
keywords = r'/kaggle/input/the-movies-dataset/keywords.csv'
links = r'/kaggle/input/the-movies-dataset/links_small.csv'
movies_metadata = r'/kaggle/input/the-movies-dataset/movies_metadata.csv'
smd = movie_feature(movies_metadata, links, credits_, keywords, more_weight_on='director')
# print(smd.columns)

In [33]:
smd["description"] = smd["description"] + " " \
                    + smd['keywords'].apply(lambda x: " ".join(map(str, x))) + " " \
                    + smd['production_companies'].apply(lambda x: " ".join(map(str, x)))

In [34]:
smd

,id,movieId,title,genres,description,keywords,cast,director,spoken_languages,production_companies,production_countries,popularity,year,vote_average,vote_count,wr
0,862,1,Toy Story,"[Animation, Comedy, Family]","Led by Woody, Andy's toys live happily in his ...","[jealousi, toy, boy, friendship, friend, rival...","[tomhanks, timallen, donrickles]","[johnlasseter, johnlasseter, johnlasseter]",[English],[Pixar Animation Studios],[United States of America],21.946943,1995,7,5415,6.869770
1,8844,2,Jumanji,"[Adventure, Fantasy, Family]",When siblings Judy and Peter discover an encha...,"[boardgam, disappear, basedonchildren'sbook, n...","[robinwilliams, jonathanhyde, kirstendunst]","[joejohnston, joejohnston, joejohnston]","[English, Français]","[TriStar Pictures, Teitler Film, Interscope Co...",[United States of America],17.015539,1995,6,2413,5.884891
2,15602,3,Grumpier Old Men,"[Romance, Comedy]",A family wedding reignites the ancient feud be...,"[fish, bestfriend, duringcreditssting]","[waltermatthau, jacklemmon, ann-margret]","[howarddeutch, howarddeutch, howarddeutch]",[English],"[Warner Bros., Lancaster Gate]",[United States of America],11.712900,1995,6,92,5.376968
3,31357,4,Waiting to Exhale,"[Comedy, Drama, Romance]","Cheated on, mistreated and stepped on, the wom...","[basedonnovel, interracialrelationship, single...","[whitneyhouston, angelabassett, lorettadevine]","[forestwhitaker, forestwhitaker, forestwhitaker]",[English],[Twentieth Century Fox Film Corporation],[United States of America],3.859495,1995,6,34,5.299755
4,11862,5,Father of the Bride Part II,[Comedy],Just when George Banks has recovered from his ...,"[babi, midlifecrisi, confid, age, daughter, mo...","[stevemartin, dianekeaton, martinshort]","[charlesshyer, charlesshyer, charlesshyer]",[English],"[Sandollar Productions, Touchstone Pictures]",[United States of America],8.387519,1995,5,173,5.175099
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9214,159550,161944,The Last Brickmaker in America,[Drama],A man must cope with the loss of his wife and ...,[friendship],"[sidneypoitier, wendycrewson, jayo.sanders]","[greggchampion, greggchampion, greggchampion]",[],[Nasser Entertainment],[United States of America],0.038998,2001,7,1,5.248931
9215,392572,162542,Rustom,"[Thriller, Romance]","Rustom Pavri, an honourable officer of the Ind...",[bollywood],"[akshaykumar, ileanad'cruz, eshagupta]","[tinusureshdesai, tinusureshdesai, tinusureshd...",[हिन्दी],[KriArj Entertainment],[India],7.333139,2016,7,25,5.340490
9216,402672,162672,Mohenjo Daro,"[Adventure, Drama, History, Romance]","Village lad Sarman is drawn to big, bad Mohenj...",[bollywood],"[hrithikroshan, poojahegde, kabirbedi]","[ashutoshgowariker, ashutoshgowariker, ashutos...",[हिन्दी],"[UTV Motion Pictures, Ashutosh Gowariker Produ...",[India],1.423358,2016,6,26,5.287576
9217,315011,163056,Shin Godzilla,"[Action, Adventure, Drama, Horror, Science Fic...",From the mind behind Evangelion comes a hit la...,"[monster, godzilla, giantmonst, destruct, kaiju]","[hirokihasegawa, yutakatakenouchi, satomiishih...","[hideakianno, hideakianno, hideakianno]","[Italiano, Deutsch, English, 日本語]","[Cine Bazar, Toho Pictures]",[Japan],9.285519,2016,6,152,5.440760


In [5]:
smd["description"][0]

"Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences. jealousi toy boy friendship friend rivalri boynextdoor newtoy toycomestolif Pixar Animation Studios"

In [6]:
smd = smd[['id', 'movieId', 'title', 'description']]
smd = smd.drop_duplicates()

In [7]:
smd

,id,movieId,title,description
0,862,1,Toy Story,"Led by Woody, Andy's toys live happily in his ..."
1,8844,2,Jumanji,When siblings Judy and Peter discover an encha...
2,15602,3,Grumpier Old Men,A family wedding reignites the ancient feud be...
3,31357,4,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom..."
4,11862,5,Father of the Bride Part II,Just when George Banks has recovered from his ...
...,...,...,...,...
9214,159550,161944,The Last Brickmaker in America,A man must cope with the loss of his wife and ...
9215,392572,162542,Rustom,"Rustom Pavri, an honourable officer of the Ind..."
9216,402672,162672,Mohenjo Daro,"Village lad Sarman is drawn to big, bad Mohenj..."
9217,315011,163056,Shin Godzilla,From the mind behind Evangelion comes a hit la...


### TF-IDF

In [8]:
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(smd['description'])
tfidf_matrix

<9082x351186 sparse matrix of type '<class 'numpy.float64'>'
	with 751859 stored elements in Compressed Sparse Row format>

In [9]:
tfidf_matrix.todense()

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

In [10]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)


In [11]:
cosine_sim.shape

(9082, 9082)

In [12]:
smd = smd.reset_index()
smd = smd.drop(columns='index')
movie_ids = smd['movieId']
indices = pd.Series(smd.index, index=smd['movieId'])

## Evaluate

In [13]:
train_df = pd.read_csv('/kaggle/input/train-set-from-ratings/train_set.csv')

In [18]:
train_df

,userId,movieId,rating,timestamp
0,431,2863,4.0,1165548515
1,571,7173,2.0,1334343358
2,77,223,4.5,1163004353
3,580,1032,4.0,1165291033
4,624,1221,5.0,1019124147
...,...,...,...,...
74998,547,5810,3.0,1415444349
74999,418,1835,4.0,1132180632
75000,5,33679,4.0,1163374517
75001,358,905,5.0,957479957


In [44]:
def get_top_n_similar(recommended, movie, cosine_sim, n_rec = 10):
    try:
        idx = indices[movie]
    except:
        # In case movie is not available in cosine_sim (or not in metadata)
        pass
    else:
        # top 10 most similar
        sim_scores = list(enumerate(cosine_sim[idx]))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        sim_scores = sim_scores[1:n_rec+1]

        for movie_rec in sim_scores:
            # change index -> movieId
            movie_recommended = (movie_ids.iloc[movie_rec[0]], movie_rec[1])
            recommended.append(movie_recommended)
    return recommended

In [45]:
def get_recommendations(user_id, cosine_sim, n_rec = 10):
    movies_id = train_df[train_df.userId == user_id].movieId 
    recommended = []
    for movie in movies_id:
        recommended = get_top_n_similar(recommended, movie, cosine_sim, n_rec = 10)
            
    # create a DataFrame to store all movies recommended            
    recommended = pd.DataFrame(recommended)  
    recommended.columns = ['movieId', 'sim']
    recommended = recommended.sort_values('sim')[::-1].head(n_rec)
    recommended['userId'] = user_id
    return recommended.drop(columns='sim')

In [46]:
def full_recommend(df, cosine_sim):
    rec_df = pd.DataFrame()
    for user in df.userId.value_counts().index:
        rec_df = pd.concat([rec_df, get_recommendations(user, cosine_sim)])
    return rec_df

### Example

In [47]:
train_df

,userId,movieId,rating,timestamp
0,431,2863,4.0,1165548515
1,571,7173,2.0,1334343358
2,77,223,4.5,1163004353
3,580,1032,4.0,1165291033
4,624,1221,5.0,1019124147
...,...,...,...,...
74998,547,5810,3.0,1415444349
74999,418,1835,4.0,1132180632
75000,5,33679,4.0,1163374517
75001,358,905,5.0,957479957


In [48]:
smd[smd.title == 'The Dark Knight']

,id,movieId,title,genres,description,keywords,cast,director,spoken_languages,production_companies,production_countries,popularity,year,vote_average,vote_count,wr
6981,155,58559,The Dark Knight,"[Drama, Action, Crime, Thriller]",Batman raises the stakes in his war on crime. ...,"[dccomic, crimefight, secretident, scarecrow, ...","[christianbale, michaelcaine, heathledger]","[christophernolan, christophernolan, christoph...","[English, 普通话]","[DC Comics, Legendary Pictures, Warner Bros., ...","[United Kingdom, United States of America]",123.167259,2008,8,12269,7.905871


In [68]:
recommend_for_film = get_top_n_similar([], 58559, cosine_sim)
recommend_for_film

[(91529, 0.2403092846334627),
 (33794, 0.19998023247875543),
 (79274, 0.17612768921909933),
 (1377, 0.16201857818088006),
 (99813, 0.15794657062511983),
 (136864, 0.1196075336144598),
 (153, 0.11887024524358365),
 (98124, 0.11743012532451377),
 (103042, 0.11023891189771333),
 (3213, 0.10993996797813119)]

In [73]:
smd[smd["movieId"].isin(pd.DataFrame(get_top_n_similar([], 58559, cosine_sim))[0])]

,id,movieId,title,genres,description,keywords,cast,director,spoken_languages,production_companies,production_countries,popularity,year,vote_average,vote_count,wr
132,414,153,Batman Forever,"[Action, Crime, Fantasy]",The Dark Knight of Gotham City confronts a das...,"[riddl, dccomic, rose, gothamc, partner, super...","[valkilmer, tommyleejones, jimcarrey]","[joelschumacher, joelschumacher, joelschumacher]",[English],"[Warner Bros., Polygram Filmed Entertainment]","[United Kingdom, United States of America]",13.321354,1995,5,1529,5.054144
1134,364,1377,Batman Returns,"[Action, Fantasy]","Having defeated the Joker, Batman now faces th...","[holiday, corrupt, doublelif, dccomic, crimefi...","[michaelkeaton, dannydevito, michellepfeiffer]","[timburton, timburton, timburton]",[English],"[PolyGram Filmed Entertainment, Warner Bros.]","[United Kingdom, United States of America]",15.001681,1992,6,1706,5.846862
2599,14919,3213,Batman: Mask of the Phantasm,"[Action, Adventure, Animation, Family]",An old flame of Bruce Wayne's strolls into tow...,"[dccomic, basedoncom, superpow]","[kevinconroy, markhamill, danadelany]","[brucetimm, brucetimm, brucetimm]","[Český, English]","[DC Comics, Warner Bros.]",[United States of America],7.291140,1993,7,218,5.831726
6218,272,33794,Batman Begins,"[Action, Crime, Drama]","Driven by tragedy, billionaire Bruce Wayne ded...","[himalaya, martialart, dccomic, crimefight, se...","[christianbale, michaelcaine, liamneeson]","[christophernolan, christophernolan, christoph...","[English, اردو, 普通话]","[DC Comics, Legendary Pictures, Warner Bros., ...","[United Kingdom, United States of America]",28.505341,2005,7,7511,6.904127
7659,40662,79274,Batman: Under the Red Hood,"[Action, Animation]",Batman faces his ultimate challenge as the mys...,"[martialart, dccomic, vigilant, joker, superhe...","[brucegreenwood, jensenackles, neilpatrickharris]","[brandonvietti, brandonvietti, brandonvietti]",[English],"[DC Comics, Warner Bros. Animation]",[United States of America],7.039325,2010,7,459,6.147016
8031,49026,91529,The Dark Knight Rises,"[Action, Crime, Drama, Thriller]",Following the death of District Attorney Harve...,"[dccomic, crimefight, terrorist, secretident, ...","[christianbale, michaelcaine, garyoldman]","[christophernolan, christophernolan, christoph...",[English],"[Legendary Pictures, Warner Bros., DC Entertai...",[United States of America],20.582580,2012,7,9263,6.921448
8265,123025,98124,"Batman: The Dark Knight Returns, Part 1","[Action, Animation]",Batman has not been seen for ten years. A new ...,"[dccomic, futur, basedongraphicnovel, dystopic...","[peterweller, arielwinter, davidselby]","[jayoliva, jayoliva, jayoliva]","[English, Français]","[DC Comics, Warner Bros. Animation, Warner Pre...",[United States of America],12.107182,2012,7,410,6.097494
8334,142061,99813,"Batman: The Dark Knight Returns, Part 2","[Action, Animation]",Batman has stopped the reign of terror that Th...,"[dccomic, futur, joker, robin, basedongraphicn...","[peterweller, arielwinter, davidselby]","[jayoliva, jayoliva, jayoliva]",[English],"[DC Comics, Warner Bros. Animation, Warner Pre...",[United States of America],12.576611,2013,7,426,6.114285
8419,49521,103042,Man of Steel,"[Action, Adventure, Fantasy, Science Fiction]",A young boy learns that he has extraordinary p...,"[savingtheworld, dccomic, superhero, basedonco...","[henrycavill, amyadams, michaelshannon]","[zacksnyder, zacksnyder, zacksnyder]",[English],"[Legendary Pictures, Warner Bros., DC Entertai...","[United Kingdom, United States of America]",18.538834,2013,6,6462,5.952478
9024,209112,136864,Batman v Superman: Dawn of Justice,"[Action, Adventure, Fantasy]",Fearing the actions of a god-like Super Hero l...,"[dccomic, vigilant, superhero, basedoncom, rev...","[benaffleck, henrycavill, galgadot]","[zacksnyder, zacksnyder, zacksnyder]",[English],"[DC Comics, Atlas Entertainment, Warner Bros.,...",[United States of America],31.435879,2016,5,7189,5.013943


In [ ]:
smd[smd["movieId"].isin(pd.DataFrame(get_top_n_similar([], 58559, cosine_sim))[0])]

### Count Vectorize

In [74]:
count_vec = CountVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
count_vec_matrix = count_vec.fit_transform(smd['description'])
count_vec_matrix

<9219x351186 sparse matrix of type '<class 'numpy.int64'>'
	with 764535 stored elements in Compressed Sparse Row format>

In [75]:
count_cosine_sim = cosine_similarity(count_vec_matrix, count_vec_matrix)
count_cosine_sim

array([[1.        , 0.0092692 , 0.        , ..., 0.        , 0.        ,
        0.00848036],
       [0.0092692 , 1.        , 0.01994324, ..., 0.01032575, 0.01769912,
        0.01619287],
       [0.        , 0.01994324, 1.        , ..., 0.01163499, 0.00997162,
        0.        ],
       ...,
       [0.        , 0.01032575, 0.01163499, ..., 1.        , 0.01032575,
        0.        ],
       [0.        , 0.01769912, 0.00997162, ..., 0.01032575, 1.        ,
        0.00809644],
       [0.00848036, 0.01619287, 0.        , ..., 0.        , 0.00809644,
        1.        ]])

In [77]:
smd[smd["movieId"].isin(pd.DataFrame(get_top_n_similar([], 58559, count_cosine_sim))[0])]

,id,movieId,title,genres,description,keywords,cast,director,spoken_languages,production_companies,production_countries,popularity,year,vote_average,vote_count,wr
261,18220,291,Poison Ivy II: Lily,"[Drama, Romance, Thriller]",A young and naive college art student becomes ...,"[sex, obsess, diari, murder, student, lust, ar...","[alyssamilano, johnathonschaech, xanderberkeley]","[annegoursaud, annegoursaud, annegoursaud]",[English],"[New Line Cinema, CineTel Pictures, MG Enterta...",[United States of America],6.448039,1996,4,32,5.159410
710,9303,866,Bound,"[Crime, Drama, Romance, Thriller]","Corky, a tough female ex con and her lover Vio...","[prison, women'sprison, suspens, mafia, lesbia...","[jennifertilly, ginagershon, joepantoliano]","[lanawachowski, lanawachowski, lanawachowski]",[English],"[Dino De Laurentiis Company, Spelling Films]",[United States of America],12.449574,1996,6,203,5.485534
1842,9713,2306,Holy Man,"[Drama, Comedy]",Eddie Murphy stars as an over-the-top televisi...,"[salesclerk, televis, tvrat, guru, televisionp...","[eddiemurphy, jeffgoldblum, kellypreston]","[stephenherek, stephenherek, stephenherek]",[English],"[Caravan Pictures, Touchstone Pictures]",[United States of America],5.403617,1998,4,113,4.987724
3877,31064,4965,The Business of Strangers,"[Drama, Thriller]","Julie Styron thinks she is going to be fired, ...","[rape, sex, airport, author, assist, power, re...","[juliastiles, stockardchanning, frederickweller]","[patrickstettner, patrickstettner, patrickstet...",[English],"[i5 Films, Headquarters]",[United States of America],2.239112,2001,5,19,5.234625
4696,10243,6448,The Flight of the Phoenix,"[Adventure, Drama]",A cargo plane goes down in a sandstorm in the ...,"[thirst, sahara, crew, airplanecrash]","[jamesstewart, richardattenborough, peterfinch]","[robertaldrich, robertaldrich, robertaldrich]",[English],"[Twentieth Century Fox Film Corporation, Assoc...",[United States of America],4.570894,1965,7,70,5.488661
5702,163,8984,Ocean's Twelve,"[Thriller, Crime]",Danny Ocean reunites with his old flame and th...,"[sequel, goldenegg]","[georgeclooney, bradpitt, catherinezeta-jones]","[stevensoderbergh, stevensoderbergh, stevensod...","[Français, Italiano, Nederlands, 普通话, English]","[Village Roadshow Pictures, Section Eight, Jer...",[United States of America],11.869347,2004,6,2169,5.874101
6567,9988,47640,Beerfest,[Comedy],During a trip to Germany to scatter their gran...,"[germani, competit, prostitut, alcohol, beer]","[erikstolhanske, jaychandrasekhar, stevelemme]","[jaychandrasekhar, jaychandrasekhar, jaychandr...","[English, Deutsch]",[Broken Lizard Industries],[United States of America],7.481125,2006,5,173,5.175099
6858,4538,55269,The Darjeeling Limited,"[Adventure, Drama, Comedy]",Three American brothers who have not spoken to...,"[brotherbrotherrelationship, dysfunctionalfami...","[owenwilson, jasonschwartzman, adrienbrody]","[wesanderson, wesanderson, wesanderson]","[English, हिन्दी, Deutsch, ]","[Fox Searchlight Pictures, Scott Rudin Product...",[United States of America],6.493474,2007,7,876,6.418538
7063,13353,60487,"It's the Great Pumpkin, Charlie Brown","[Family, Animation]","This classic ""Peanuts"" tale focuses on the thu...",[holiday],"[christophershea, sallydryer, kathysteinberg]","[billmelendez, billmelendez, billmelendez]",[English],[Warner Bros. Home Video],[United States of America],6.588367,1966,7,84,5.529508
8453,59558,103813,Miami Connection,"[Action, Crime]",In a world where ninjas dream of being rock st...,"[ninja, blood, fallingfromheight]","[y.k.kim, vincenthirsch, josephdiamand]","[y.k.kim, y.k.kim, y.k.kim]",[English],[Drafthouse Films],"[United States of America, Hong Kong]",1.972494,1987,6,28,5.290660
